In [2]:
import pandas  as pd
import numpy as np
import os
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
nb = MultinomialNB()
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.utils import class_weight
from sklearn.svm import SVC
from scipy.sparse import hstack, vstack
from nltk.corpus import stopwords  
#nltk.download('stopwords') 
import pickle
import datetime
from dateutil import parser
import sys
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score ,accuracy_score,confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#english_stop=["a","about","above","after","again","against","all","am","an","and","any","are","aren't","as","at","be","because","been","before","being","below","between","both","but","by","during","each","few","for","from","further","having","he","he'd","he'll","he's","her","here","here's","hers","herself","him","himself","his","how","how's","i","i'd","i'll","i'm","i've","if","in","into","is","isn't","it","it's","its","itself","let's","me","more","most","mustn't","my","myself","no","nor","not","of","off","on","once","only","or","other","ought","our","ours","ourselves","out","over","own","same","shan't","she","she'd","she'll","she's","should","shouldn't","so","some","such","than","that","that's","the","their","theirs","them","themselves","then","there","there's","these","they","they'd","they'll","they're","they've","this","those","through","to","too","under","until","up","very","was","we","we'd","we'll","we're","we've","were","weren't","what","what's","when","when's","where","where's","which","while","who","who's","whom","why","why's","with","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves"]
#self_stop=["end", "dell","del", "support","chromebook", "chrome book", "bad","good", "doesnt", "isnt", "latitude", "power edge", "power edge", "working", "issue", "SR", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "xx", "xxx", "-", "_", ".", ",", "iw", "micro"]
#stopwords = english_stop+ self_stop
#stopwords = list(map(lambda x: x.upper(), english_stop))


In [ ]:
#re.sub(r'[^\w]', ' ', data["NOTE_TXT"])

In [3]:
data = pd.read_excel("C:\\Users\\Abhishek_Prasar\\Documents\\My Received Files\\train_29_march_preprocessed.xlsx")
data.head()

,Unnamed: 0,FWEEK,FMONTH,FQUARTER,FYEAR,DPS_NUM,PRODUCT_DESC,BRAND_DESC,DPSTYPE,DPSDATE,...,Transactional_TSL,Dispatch cancellation time,Day Difference b/w DPS creation & cancellation,Can cancellation reason be determined using Comments from Vendor,CANCELLED_BY_AGENT,CANCELLED_BY_MGRNAME,CANCELLED_BY_AMGRNAME,Language,Reason_for_Cancellation_COE,NOTES_COE
0,0,201935,2019M09,2019Q03,2019,370999887,OptiPlex Desktops,OPTIPLEX 7060,NBU,2018-10-04 00:27:28.0000000,...,Kelly Truax,2018-10-13 21:47:38.000000,9.0,Yes,"Ogan, Jason","Smith, Frederick","Haworth, Scott",English,TS DUPLICATE DPS CREATED,BY DISPATCH NO LONGER NEED
1,1,201936,2019M09,2019Q03,2019,372031241,Chrome,CHROMEBOOK 11 3180,RTS,2018-10-12 15:29:56.0000000,...,NaN,2018-11-06 15:48:02.000000,25.0,Yes,"Suderman, Kevin","Fite, David","Haworth, Scott",English,TS DUPLICATE DPS CREATED,BY WA ALREADI SHIP BACK CUSTOM SHOW A PICTUR O...
2,2,201937,2019M09,2019Q03,2019,17372358148,OptiPlex Desktops,OPTIPLEX 3040,NBD,2018-10-16 13:53:11.0000000,...,Kelly Truax,2018-10-19 14:22:33.000000,3.0,Yes,"Bakowski, Mateusz","FakeLast, FakeFirst","FakeLast, FakeFirst",English,TS DUPLICATE DPS CREATED,SEE COMMENT WRONG EMAIL LOG ISSU NO POWER HARD...
3,3,201935,2019M09,2019Q03,2019,385610825,Latitude,LATITUDE 5550,RTS,2018-10-03 19:33:09.0000000,...,NaN,2018-10-04 16:12:46.000000,1.0,Yes,"Kiper, Kyle","Windish, Daniel","Hernandez, Victor",English,TS DUPLICATE DPS CREATED,ALREADI RECREAT THI ONE
4,4,201935,2019M09,2019Q03,2019,370861455,Alienware Notebooks,ALIENWARE 17 R3,RPQ,2018-10-03 02:02:01.0000000,...,Denis Kelly,2018-10-10 02:42:44.000000,7.0,Yes,"WalkToManifest, WalkToManifest","Manager, TBD","Manager, TBD",English,TS DUPLICATE DPS CREATED,BY QLX CANCEL BY DSP DSP WORK


In [ ]:
"""
colon = '\uff1a'
semicolon = '\uff1b'
comma = '\uff0c'
enum_comma = '\u3001'
left_parenthesis = '\uff08'
right_parenthesis = '\uff09'
period = '\uff0e'
japanese_square_bullet = '\u25a0'
japanese_diamond_bullet = '\u25c6'
"""

In [ ]:
#data.NOTE_TXT = list(map(lambda x: x.replace(comma,",").replace(enum_comma, ",").replace(period, ".").replace(semicolon,";").replace(colon,":").replace(left_parenthesis,"(" ).replace(right_parenthesis,")"), data.NOTE_TXT.astype(str)))

In [4]:

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data['Final_Category_Encoded'] = lb.fit_transform(data['Reason_for_Cancellation_COE'])
data.head()


,Unnamed: 0,FWEEK,FMONTH,FQUARTER,FYEAR,DPS_NUM,PRODUCT_DESC,BRAND_DESC,DPSTYPE,DPSDATE,...,Dispatch cancellation time,Day Difference b/w DPS creation & cancellation,Can cancellation reason be determined using Comments from Vendor,CANCELLED_BY_AGENT,CANCELLED_BY_MGRNAME,CANCELLED_BY_AMGRNAME,Language,Reason_for_Cancellation_COE,NOTES_COE,Final_Category_Encoded
0,0,201935,2019M09,2019Q03,2019,370999887,OptiPlex Desktops,OPTIPLEX 7060,NBU,2018-10-04 00:27:28.0000000,...,2018-10-13 21:47:38.000000,9.0,Yes,"Ogan, Jason","Smith, Frederick","Haworth, Scott",English,TS DUPLICATE DPS CREATED,BY DISPATCH NO LONGER NEED,18
1,1,201936,2019M09,2019Q03,2019,372031241,Chrome,CHROMEBOOK 11 3180,RTS,2018-10-12 15:29:56.0000000,...,2018-11-06 15:48:02.000000,25.0,Yes,"Suderman, Kevin","Fite, David","Haworth, Scott",English,TS DUPLICATE DPS CREATED,BY WA ALREADI SHIP BACK CUSTOM SHOW A PICTUR O...,18
2,2,201937,2019M09,2019Q03,2019,17372358148,OptiPlex Desktops,OPTIPLEX 3040,NBD,2018-10-16 13:53:11.0000000,...,2018-10-19 14:22:33.000000,3.0,Yes,"Bakowski, Mateusz","FakeLast, FakeFirst","FakeLast, FakeFirst",English,TS DUPLICATE DPS CREATED,SEE COMMENT WRONG EMAIL LOG ISSU NO POWER HARD...,18
3,3,201935,2019M09,2019Q03,2019,385610825,Latitude,LATITUDE 5550,RTS,2018-10-03 19:33:09.0000000,...,2018-10-04 16:12:46.000000,1.0,Yes,"Kiper, Kyle","Windish, Daniel","Hernandez, Victor",English,TS DUPLICATE DPS CREATED,ALREADI RECREAT THI ONE,18
4,4,201935,2019M09,2019Q03,2019,370861455,Alienware Notebooks,ALIENWARE 17 R3,RPQ,2018-10-03 02:02:01.0000000,...,2018-10-10 02:42:44.000000,7.0,Yes,"WalkToManifest, WalkToManifest","Manager, TBD","Manager, TBD",English,TS DUPLICATE DPS CREATED,BY QLX CANCEL BY DSP DSP WORK,18


In [ ]:
data["NOTES_COE"] = data["NOTES_COE"].str.upper()

In [ ]:
lb_name_mapping = dict(zip(lb.classes_, lb.transform(lb.classes_)))


In [ ]:
#Code
"""
def first_k(s: str, k=200) -> str:
    s = str(s)  # just in case something like NaN tries to sneak in there
    first_words = s.split()[:k]
    return ' '.join(first_words)
"""

In [ ]:

#data["NOTE_TXT"] = data["NOTE_TXT"].apply(first_k)

In [ ]:
#data = data.reset_index(drop = True)

In [ ]:
data["NOTES_COE"] = data["NOTES_COE"].astype(str)
data["NOTES_COE"] = [x.replace(':',' ') for x in data["NOTES_COE"]]

In [ ]:
#Removing Punctuations
data["NOTES_COE"]  = data["NOTES_COE"] .str.replace('[^\w\s]','')
data["NOTES_COE"] .head()

In [ ]:
"""
from nltk.corpus import stopwords
stop = stopwords.words('english')
data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data["NOTES_COE"].head()
"""

In [ ]:
"""
#Rare Word Removal
freq = pd.Series(' '.join(data["NOTES_COE"]).split()).value_counts()[-20:]
"""

In [ ]:
"""
freq = list(freq.index)
data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data["NOTES_COE"].head()
"""

In [ ]:
#Spelling correction
#from textblob import TextBlob
#data["NOTES_COE"].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
#Tokenizing
#from textblob import TextBlob
#TextBlob(data["NOTES_COE"].astype(str)).words

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
data["NOTES_COE"].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
#Lemmatization
from textblob import Word
data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data["NOTES_COE"].head()

In [ ]:
"""
word_checker = pd.read_excel("C:\\Users\\Abhishek_Prasar\\Desktop\\data\\Shreejit.xlsx",sheet_name="Sheet1")

word_checker["NOTE_TXT"] = word_checker["NOTE_TXT"].str.upper()
word_checker["NOTE_TXT"] = [x.replace(':', '  ') for x in word_checker["NOTE_TXT"]]
#Removing Punctuations
word_checker["NOTE_TXT"] = word_checker["NOTE_TXT"].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
stop = stopwords.words('english')
word_checker["NOTE_TXT"] = word_checker["NOTE_TXT"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Rare Word Removal
freq = pd.Series(' '.join(word_checker["NOTE_TXT"]).split()).value_counts()[-20:]
#Rare Word Removal
freq = pd.Series(' '.join(word_checker["NOTE_TXT"]).split()).value_counts()[-20:]

freq = list(freq.index)
word_checker["NOTE_TXT"] = word_checker["NOTE_TXT"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
word_checker["NOTE_TXT"].head()

#Lemmatization
from textblob import Word
word_checker["NOTE_TXT"] = word_checker["NOTE_TXT"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
word_checker["NOTE_TXT"].head()
"""

In [ ]:
#Unique Find Finder
"""
import re
 

regex = re.compile(r"[\W_]")
unique_words = set()
for line in word_checker["NOTE_TXT"]:
    line = line.rstrip('\n')
    words = regex.split(line)
    for w in words:
        if w != "":
            w = w.upper()
            unique_words.add(w)
 
print(unique_words)
unique_words = list(unique_words)

"""


In [ ]:
#data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x in unique_words))
#data["NOTES_COE"].head()

In [ ]:
#data.to_excel("D://my_data//for_keras.xlsx")

In [5]:
valid = pd.read_excel("C:\\Users\\Abhishek_Prasar\\Documents\\My Received Files\\test_29_march_preprocessed.xlsx")
valid[['COE_NOTES','Reason_for_Cancellation_ENT']].head()

,COE_NOTES,Reason_for_Cancellation_ENT
0,Cancellation requested by: QLX. Reason: CANCEL...,CUSTOMER NOT AVAILABLE/REACHABLE/ SYSTEM NOT A...
1,Cancellation requested by: Dell. Reason:Aging ...,CUSTOMER DENIED SERVICE
2,Cancellation requested by: QLX. Reason: CANCEL...,LOGISTICS PNA/CNS
3,Cancellation requested by: Dell. Reason:Incide...,CUSTOMER REQUESTED DIFFERENT DATE
4,Cancellation requested by: QLX. Reason: CANCEL...,CUSTOMER REQUESTED DIFFERENT DATE


In [6]:
valid['Final_Category_Encoded1'] = lb.transform(valid['Reason_for_Cancellation_ENT'])

In [ ]:
"""
valid["COE_NOTES"] = valid["COE_NOTES"].str.upper()
valid["COE_NOTES"] = [x.replace(':', '  ') for x in valid["COE_NOTES"]]
#Removing Punctuations
valid["COE_NOTES"]  = valid["COE_NOTES"] .str.replace('[^\w\s]','')
from nltk.corpus import stopwords
stop = stopwords.words('english')
valid["COE_NOTES"] = valid["COE_NOTES"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Rare Word Removal
freq = pd.Series(' '.join(valid["COE_NOTES"]).split()).value_counts()[-20:]
#Rare Word Removal
freq = pd.Series(' '.join(valid["COE_NOTES"]).split()).value_counts()[-20:]

freq = list(freq.index)
valid["COE_NOTES"] = valid["COE_NOTES"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
valid["COE_NOTES"].head()

#Lemmatization
from textblob import Word
valid["COE_NOTES"] = valid["COE_NOTES"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
valid["COE_NOTES"].head()
"""

In [ ]:
#valid["COE_NOTES"] = valid["COE_NOTES"].apply(lambda x: " ".join(x for x in x.split() if x in unique_words))
#valid["COE_NOTES"].head()

In [ ]:
#X = data["NOTES_COE"]
#y= data['Final_Category_Encoded']

X is 1D currently because it will be passed to Vectorizer to become a 2D matrix

You must always have a 1D object so CountVectorizer can turn into a 2D object for the model to be built on


In [ ]:
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,train_size = 0.8)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
"""

In [7]:

X_train = data["NOTES_COE"]
X_test = valid["COE_NOTES"]
y_train = data["Final_Category_Encoded"]
y_test = valid["Final_Category_Encoded1"]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(4554,)
(1974,)
(4554,)
(1974,)


Split then vectorize (correct way)

We do the train/test split before the CountVectorizer to properly simulate the real world where our future data contains words we have not seen before

After training your data and chose the best model, you would then train on all of your data before predicting actual future data to maximize learning.

we are learning the vocabulary dictionary and it returns a Document-Term matrix. 

In [8]:
# 1. import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

# 2. instantiate CountVectorizer (vectorizer)
vect = CountVectorizer(stop_words=None)

# learn the 'vocabulary' of the training data (occurs in-place)
X_train_dtm = vect.fit_transform(X_train.astype(str))
X_train_dtm.shape

(4554, 6412)

In [9]:
# 3. transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test.astype(str))
X_test_dtm
#the number of columns, 4552, is the same as what we have learned above in X_train_dtm

<1974x6412 sparse matrix of type '<class 'numpy.int64'>'
	with 35284 stored elements in Compressed Sparse Row format>

TF-IDF: Finally, we can even reduce the weightage of more common words like (the, is, an etc.) which occurs in all document. This is called as TF-IDF i.e Term Frequency times inverse document frequency.

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_dtm)
#X_train_tfidf.shape

In [11]:
X_test_tfidf = tfidf.transform(X_test_dtm)
#X_test_tfidf.shape

In [12]:
#X_valid_tfidf = tfidf_transformer.transform(X_valid_dtm)
#X_valid_tfidf.shape

In [13]:
X_train_tfidf

<4554x6412 sparse matrix of type '<class 'numpy.float64'>'
	with 78758 stored elements in Compressed Sparse Row format>

In [14]:
#import nltk
#from nltk.stem.porter import PorterStemmer
#porter_stemmer = PorterStemmer()
#X_train_dtm = X_train_dtm.apply(lambda x: ''.join([porter_stemmer.stem(X_train_dtm) for X_train_dtm in x.split()])) 

In [15]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_train_balanced, y_train_balanced= ros.fit_resample(X_train_tfidf, y_train)

In [16]:
#unique, counts = np.unique(y_train_balanced, return_counts=True)
#print(list(zip(unique, counts)))

In [17]:
#X_test_balanced, y_test_balanced= ros.fit_sample(X_test_tfidf, y_test)

In [18]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty="l1")
%time lr.fit(X_train_balanced, y_train_balanced)
#make class predictions for X_test_dtm
y_pred_class = lr.predict(X_test_tfidf)
# calculate accuracy of class predictions
print("accuracy_score:",accuracy_score(y_test, y_pred_class))
# calculate AUC
#print("f1_score:",f1_score(y_test, y_pred_class,average='macro'))

Wall time: 4.29 s
accuracy_score: 0.35916919959473154


In [19]:
from sklearn.svm import LinearSVC
lrsvc = LinearSVC()
%time lrsvc.fit(X_train_dtm, y_train)
#make class predictions for X_test_dtm
y_pred_class = lrsvc.predict(X_test_tfidf)
# calculate accuracy of class predictions
print("accuracy_score:",accuracy_score(y_test, y_pred_class))
# calculate AUC
print("f1_score:",f1_score(y_test, y_pred_class,average='macro'))

Wall time: 2.92 s
accuracy_score: 0.3009118541033435
f1_score: 0.3247826307812787


In [ ]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier()
%time SGDC.fit(X_train_balanced, y_train_balanced)
#make class predictions for X_test_dtm
y_pred_class = SGDC.predict(X_test_tfidf)
# calculate accuracy of class predictions
print("accuracy_score:",accuracy_score(y_test, y_pred_class))
# calculate AUC
print("f1_score:",f1_score(y_test, y_pred_class,average='macro'))

In [ ]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier()
%time SGDC.fit(X_train_balanced,y_train_balanced)
#make class predictions for X_test_dtm
y_pred_class = SGDC.predict(X_test_tfidf)
# calculate accuracy of class predictions
print("accuracy_score:",accuracy_score(y_test, y_pred_class))
# calculate AUC
print("f1_score:",f1_score(y_test, y_pred_class,average='macro'))

In [ ]:
"""
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                      ('lr', LogisticRegression()),
 ])
 """

In [ ]:
"""
import nltk
nltk.download()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                    ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
])
text_mnb_stemmed = text_mnb_stemmed.fit(X_train_balanced,y_train_balanced)
predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test_dtm)
np.mean(predicted_mnb_stemmed == twenty_test.target)
"""

In [ ]:
from sklearn.model_selection import ShuffleSplit

X1 = data.NOTES_COE
y1 = data.Final_Category_Encoded

ss = ShuffleSplit(n_splits=10, test_size=0.2)

from sklearn.linear_model import LogisticRegression
lr1 = LogisticRegression()

from imblearn.over_sampling import RandomOverSampler
rs = RandomOverSampler()

accs = []
f1s = []


for train_index, test_index in ss.split(X):
    
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    
    # Fit vectorizer and transform X train, then transform X test
    X_train_vect = vect.fit_transform(X_train.astype(str))
    X_test_vect = vect.transform(X_test.astype(str))
    
    # tfidf
    X_train_with_tfidf = tfidf.fit_transform(X_train_vect)
    X_test_with_tfidf = tfidf.transform(X_test_vect)
    
    
    # Oversample
    X_train_res, y_train_res = rs.fit_sample(X_train_with_tfidf, y_train)
    
    # #Fitting and Predictions
    
    lr1.fit(X_train_res, y_train_res)
    y_pred = lr1.predict(X_test_with_tfidf)
    
    # Determine test set accuracy and f1 score on this fold using the true y labels and predicted y labels
    accs.append(accuracy_score(y_test, y_pred))
    #f1s.append(f1_score(y_test, y_pred,average='macro'))

    
print("\nAverage accuracy across folds: {:.2f}%".format(sum(accs) / len(accs) * 100))
print("\nAverage F1 score across folds: {:.2f}%".format(sum(f1s) / len(f1s) * 100))


In [ ]:
from sklearn.model_selection import ShuffleSplit

X1 = data.NOTES_COE
y1 = data.Final_Category_Encoded

ss = ShuffleSplit(n_splits=10, test_size=0.2)

from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier(penalty="elasticnet")

from imblearn.over_sampling import RandomOverSampler
rs = RandomOverSampler()

accs = []
f1s = []


for train_index, test_index in ss.split(X):
    
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    
    # Fit vectorizer and transform X train, then transform X test
    X_train_vect = vect.fit_transform(X_train.astype(str))
    X_test_vect = vect.transform(X_test.astype(str))
    
    # tfidf
    X_train_with_tfidf = tfidf.fit_transform(X_train_vect)
    X_test_with_tfidf = tfidf.transform(X_test_vect)
    
    
    # Oversample
    X_train_res, y_train_res = rs.fit_sample(X_train_with_tfidf, y_train)
    
    # #Fitting and Predictions
    
    SGDC.fit(X_train_res, y_train_res)
    y_pred = SGDC.predict(X_test_with_tfidf)
    
    # Determine test set accuracy and f1 score on this fold using the true y labels and predicted y labels
    accs.append(accuracy_score(y_test, y_pred))
    f1s.append(f1_score(y_test, y_pred,average='macro'))

    
print("\nAverage accuracy across folds: {:.2f}%".format(sum(accs) / len(accs) * 100))
print("\nAverage F1 score across folds: {:.2f}%".format(sum(f1s) / len(f1s) * 100))


In [ ]:
from sklearn.model_selection import ShuffleSplit

X1 = data.NOTES_COE
y1 = data.Final_Category_Encoded

ss = ShuffleSplit(n_splits=5, test_size=0.2)

from sklearn.svm import LinearSVC
lrsvc  = LinearSVC()

from imblearn.over_sampling import RandomOverSampler
rs = RandomOverSampler()

accs = []
f1s = []


for train_index, test_index in ss.split(X):
    
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    
    # Fit vectorizer and transform X train, then transform X test
    X_train_vect = vect.fit_transform(X_train.astype(str))
    X_test_vect = vect.transform(X_test.astype(str))
    
      # tfidf
    X_train_with_tfidf = tfidf.fit_transform(X_train_vect)
    X_test_with_tfidf = tfidf.transform(X_test_vect)
    
    
    # Oversample
    X_train_res, y_train_res = rs.fit_sample(X_train_with_tfidf, y_train)
    
    
    lrsvc.fit(X_train_res, y_train_res)
    y_pred = lrsvc.predict(X_test_with_tfidf)
    
    
    # Determine test set accuracy and f1 score on this fold using the true y labels and predicted y labels
    accs.append(accuracy_score(y_test, y_pred))
    f1s.append(f1_score(y_test, y_pred,average='macro'))
    
    #predictions
    
    

    
print("\nAverage accuracy across folds: {:.2f}%".format(sum(accs) / len(accs) * 100))
print("\nAverage F1 score across folds: {:.2f}%".format(sum(f1s) / len(f1s) * 100))


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = { "penalty":['l2','l1'],          
              'C': [0.0001,.001,.01,.1,1]}

In [ ]:
gs_clf = GridSearchCV(lr, parameters, n_jobs=-1,cv= ss)
gs_clf = gs_clf.fit(X_train_vect, y_train)

In [ ]:
valid = pd.read_excel("D://final_data//Data_W45_28_March_lang.xlsx")

In [ ]:
valid.head()

In [ ]:
valid["COE_NOTES"] = lb.transform(valid["Reason_for_Cancellation"])

In [ ]:
valid["COE_NOTES"] = valid["COE_NOTES"].astype(str).str.upper()
valid["COE_NOTES"].head()

In [ ]:
"""
from nltk.stem import PorterStemmer
port = PorterStemmer()
valid['COE_NOTES']=valid['COE_NOTES'].apply(lambda x : filter(None,x(" ")))
valid['COE_NOTES']=valid['COE_NOTES'].apply(lambda x : [port.stem(y) for y in x])
"""

In [ ]:
#Stop words removal
#from nltk.corpus import stopwords
#stop = stopwords.words('english') +self_stop
#valid['NOTE_TXT'] = valid['NOTE_TXT'].apply(lambda x: ' '.join([word for word in x() if word not in (stop)]))


In [ ]:
valid["COE_NOTES"] = valid["COE_NOTES"].astype(str)

In [ ]:
valid["COE_NOTES"] = valid["COE_NOTES"].str.upper()

In [ ]:
#Rare words removal
#freq2 = pd.Series(' '.join(valid['NOTE_TXT']).split()).value_counts()[-10:]

In [ ]:

#freq2 = list(freq2.index)
#valid['NOTE_TXT'] = valid['NOTE_TXT'].apply(lambda x: " ".join(x for x in x.split() if x not in freq2))

In [ ]:
X = valid["NOTE_TXT"].astype(str)

In [ ]:
X_vect = vect.transform(X)

In [ ]:
X_idf = tfidf.transform(X_vect)

In [ ]:
valid["predict_cat"] = lrsvc.predict(X_idf)

In [ ]:
valid.to_excel("D://final_data//out1_now_1.xlsx")